In [48]:
pip install aiohttp

Note: you may need to restart the kernel to use updated packages.


In [14]:
tiers=["IRON","BRONZE","SILVER","GOLD","PLATINUM","EMERALD","DIAMOND"]
divisions = ["I", "II", "III", "IV"]
pages=["1","2"]

In [ ]:
#https://developer.riotgames.com/apis#summoner-v4/GET_getBySummonerName
#https://developer.riotgames.com/apis#match-v5/GET_getMatchIdsByPUUID

In [23]:
import csv
import requests
import time  # Import the time module

def fetch_and_save_ranked_players(tiers, divisions, pages):
    api_key = "RGAPI-9700eac4-1e3c-4da8-92d2-d2168387e08f"  # Use your actual API key
    base_url = "https://euw1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{tier}/{division}?page={page}"
    headers = {
        "X-Riot-Token": api_key
    }
    
    file_path = '../DataProcessing/TestData/rankedPlayers.csv'
    
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['tier', 'division', 'summonerName'])  # CSV Header

        for tier in tiers:
            for division in divisions:
                for page in pages:
                    url = base_url.format(tier=tier, division=division, page=page)
                    response = requests.get(url, headers=headers)

                    # Wait for 2 seconds before making the next request
                    time.sleep(2)

                    if response.status_code == 200:
                        players = response.json()
                        for player in players:
                            if player['wins'] + player['losses'] > 20:
                                writer.writerow([player['tier'], player['rank'], player['summonerName']])
                    else:
                        print(f"Error: {response.status_code}, {response.text}")
# Note: Before running, replace 'Your_API_Key_Here' with your actual API key and ensure the file path for CSV is correct as per your environment.


In [24]:
#ONLY RUN IF NEW DATA REQUIRED
#fetch_and_save_ranked_players(tiers, divisions, pages)

In [25]:
#def get_first_100_summoner_names(file_path):
#    summoner_names = []
    
#    with open(file_path, mode='r', encoding='utf-8') as file:
#        reader = csv.DictReader(file)
#        for row in reader:
#            if len(summoner_names) < 100:
#                summoner_names.append(row['summonerName'])
#            else:
#                break
    
#    return summoner_names


In [31]:
import csv

def find_duplicate_summoner_names_with_rows(file_path):
    summoner_occurrences = {}  # Tracks summoner names and their row numbers
    duplicates_info = []  # Stores information about duplicates

    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row_number, row in enumerate(reader, start=1):  # Start counting rows from 1
            summoner_name = row['summonerName']
            if summoner_name in summoner_occurrences:
                # If the summoner name is already encountered, add the current row as a duplicate
                summoner_occurrences[summoner_name].append(row_number)
            else:
                # Otherwise, initialize the list with the current row number
                summoner_occurrences[summoner_name] = [row_number]

    # Filter out summoner names with more than one occurrence and prepare duplicate info
    for summoner_name, row_numbers in summoner_occurrences.items():
        if len(row_numbers) > 1:
            duplicates_info.append((summoner_name, row_numbers))

    return duplicates_info

# Specify the file path
#file_path = '../DataProcessing/TestData/rankedPlayers.csv'
#duplicates = find_duplicate_summoner_names_with_rows(file_path)

# Example output
#for summoner_name, rows in duplicates:
#    print(f"Duplicate: {summoner_name} found in rows: {rows}")


In [32]:
import csv

def remove_duplicate_summoner_names(file_path):
    processed_summoners = set()  # To track unique summonerNames
    unique_rows = []  # To store rows after removing duplicates

    # Read the file and filter out duplicate summonerNames
    with open(file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            if row['summonerName'] not in processed_summoners:
                processed_summoners.add(row['summonerName'])
                unique_rows.append(row)

    # Write the unique rows back to the file
    with open(file_path, mode='w', newline='', encoding='utf-8') as outfile:
        fieldnames = ['tier', 'division', 'summonerName']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(unique_rows)

# Specify the file path
#file_path = '../DataProcessing/TestData/rankedPlayers.csv'
#remove_duplicate_summoner_names(file_path)


In [34]:
def split_csv_by_tier(source_file_path):
    # Define the base folder for the new files
    base_folder = os.path.dirname(source_file_path)
    
    # Initialize a dictionary to keep track of CSV writers for each tier
    writers = {}
    files = {}
    
    with open(source_file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            tier = row['tier']
            file_name = f"{tier}_players.csv"
            file_path = os.path.join(base_folder, file_name)
            
            # Check if we already have a writer for this tier, if not create it
            if tier not in writers:
                files[tier] = open(file_path, mode='w', newline='', encoding='utf-8')
                writers[tier] = csv.DictWriter(files[tier], fieldnames=reader.fieldnames)
                writers[tier].writeheader()
            
            # Write the row to the appropriate file
            writers[tier].writerow(row)
    
    # Close all the files we've opened
    for file in files.values():
        file.close()

# Specify the source file path
#source_file_path = '../DataProcessing/TestData/rankedPlayers.csv'

# Call the function
#split_csv_by_tier(source_file_path)


In [44]:
def get_summoner_names_from_csv(file_path):
    summoner_names = []
    
    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            summoner_names.append(row['summonerName'])
    
    return summoner_names

# Specify the path to your CSV file
#file_path = '../DataProcessing/TestData/SILVER_players.csv'
#summoner_names = get_summoner_names_from_csv(file_path)
print(summoner_names)


['iCrimson', 'LilBabyMan', 'Niki324', 'Elemmë', 'ZaCARRYa', 'NankuruKaisa', 'Schmael', 'Fran1kenStein', 'OriginsOfBretzel', 'QQ ousaY', 'EDSIOT', 'KELELIOS', 'Attolar', 'Cocoα', 'Dark4422', 'UwUeha', 'AlexiaJulis', 'l Ζεύς l', 'ØωΘ', 'D1azx', 'Mestre Xiau Lin', 'CyberNate', 'Mclovín', 'Poori44', 'kronstadtgg', 'CheezWithWhine', 'Rhydeł', 'GONGAX', 'Dwenty', 'Alfer41', 'Roterheld1', 'Darkin Shaco', 'Pxngsmoke', 'tiredandh0rny', 'I WILL KICK YOU', 'CristianDy11', 'HR Aurel9922', 'NorthernLightssS', 'Caede', 'Okæææy', 'penjuan420', 'Rose Monroe', 'JetztKnallts', 'ClumsyToni', 'pretty lux main', 'HideÔnWaifu', 'HolySchneefl0cke', 'Lluis2003623', 'QUENDALL', 'KHSO', 'Schmídt', 'SSVM Plantblood', 'MOUCHTASU456', 'XDEADMAU5', 'Gonrain', 'Littlepyscho', 'Valthyrix', 'Luxary147', 'IIKiuubyII', 'DK Dęft', 'VikingTryndamere', 'Br3wz', 'Fungí Man', 'Xx55KhaZix55xX', 'EkioerIII', 'Onlyfan Rakan', 'Schneiden123', 'GayLoOoOoD', 'LxGGxs', 'Zellosia', '1L MISTER', 'yo japo', 'jim rufus', 'baηzai noodle

In [45]:
import requests
import time
import csv
import os

def update_summoner_names_with_puuid(file_path, summoner_names):
    api_key = "RGAPI-9700eac4-1e3c-4da8-92d2-d2168387e08f"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }
    summoner_to_puuid = {}

    start_time = time.time()
    last_update_time = start_time
    total_names = len(summoner_names)
    names_processed = 0

    # Fetch puuid for each summoner name
    for name in summoner_names:
        url = f"https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}"
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            summoner_to_puuid[name] = data['puuid']
        else:
            print(f"Error fetching data for {name}: {response.status_code}")
        names_processed += 1
        current_time = time.time()

        # Check if 1 minute has passed for progress update
        if current_time - last_update_time >= 60:
            print(f"Progress: {names_processed}/{total_names} summoner names processed.")
            last_update_time = current_time

        time.sleep(1.5)  # Delay to comply with rate limit
    
    # Update progress after finishing all requests
    print(f"Progress: {names_processed}/{total_names} summoner names processed. Update complete.")

    # Now, replace summoner names with puuid in the CSV
    temp_file_path = file_path + ".tmp"
    with open(file_path, mode='r', encoding='utf-8') as infile, open(temp_file_path, mode='w', newline='', encoding='utf-8') as outfile:
        reader = csv.DictReader(infile)
        writer = csv.DictWriter(outfile, fieldnames=reader.fieldnames)
        writer.writeheader()
        for row in reader:
            if row['summonerName'] in summoner_to_puuid:
                row['summonerName'] = summoner_to_puuid[row['summonerName']]
            writer.writerow(row)
    
    # Replace original file with the updated temp file
    os.replace(temp_file_path, file_path)


# Call the function
#update_summoner_names_with_puuid(file_path, summoner_names)



Progress: 30/888 summoner names processed.
Progress: 61/888 summoner names processed.
Progress: 91/888 summoner names processed.
Progress: 121/888 summoner names processed.
Progress: 151/888 summoner names processed.
Progress: 182/888 summoner names processed.
Progress: 213/888 summoner names processed.
Progress: 244/888 summoner names processed.
Progress: 275/888 summoner names processed.
Progress: 306/888 summoner names processed.
Progress: 337/888 summoner names processed.
Progress: 367/888 summoner names processed.
Progress: 397/888 summoner names processed.
Progress: 427/888 summoner names processed.
Progress: 457/888 summoner names processed.
Progress: 488/888 summoner names processed.
Progress: 519/888 summoner names processed.
Progress: 550/888 summoner names processed.
Progress: 581/888 summoner names processed.
Progress: 612/888 summoner names processed.
Progress: 643/888 summoner names processed.
Progress: 674/888 summoner names processed.
Progress: 705/888 summoner names pr

In [50]:
import os

def create_files_for_each_tier(tiers, base_folder='../DataProcessing/TestData/'):
    # Ensure the base folder exists
    os.makedirs(base_folder, exist_ok=True)
    
    for tier in tiers:
        # Construct the filename for each tier, ending with _GAMES
        filename = f"{tier}_GAMES.csv"
        file_path = os.path.join(base_folder, filename)
        
        # Create an empty file for the tier
        with open(file_path, 'w') as file:
            # Optionally, write a header or initial content here
            # For now, just creating an empty file
            pass
            
    print(f"All files have been created in {base_folder}.")

# Define the tiers
#tiers = ["BRONZE", "SILVER", "GOLD", "PLATINUM", "EMERALD", "DIAMOND"]

# Call the function to create the files
#create_files_for_each_tier(tiers)



All files have been created in ../DataProcessing/TestData/.


In [47]:
import requests
import csv
import time
import os

def fetch_matches_and_save(source_csv_path, target_csv_path):
    api_key = "RGAPI-9700eac4-1e3c-4da8-92d2-d2168387e08f"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }
    
    start_time = time.time()
    last_update_time = start_time
    summoners_processed = 0

    # Ensure the target file directory exists
    os.makedirs(os.path.dirname(target_csv_path), exist_ok=True)

    with open(source_csv_path, mode='r', encoding='utf-8') as infile, open(target_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
        csv_reader = csv.DictReader(infile)
        csv_writer = csv.writer(outfile)
        csv_writer.writerow(['MatchID'])  # Header for target CSV
        
        for row in csv_reader:
            puuid = row['summonerName']  # Assuming this column actually contains puuids
            url = f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count=20"
            response = requests.get(url, headers=headers)
            
            if response.status_code == 200:
                match_ids = response.json()
                for match_id in match_ids:
                    csv_writer.writerow([match_id])
            else:
                print(f"Error fetching matches for {puuid}: {response.status_code}")
            
            summoners_processed += 1
            current_time = time.time()
            
            # Rate limiting
            time.sleep(1.5)
            
            # Print progress every minute
            if current_time - last_update_time >= 60:
                print(f"Progress: {summoners_processed} summoners processed.")
                last_update_time = current_time

    # Final progress update
    print(f"Final Progress: {summoners_processed} summoners processed. Task complete.")

# Example usage
#source_csv_path = '../DataProcessing/TestData/IRON_players.csv'
#target_csv_path = '../DataProcessing/TestData/IRON_GAMES.csv'
#fetch_matches_and_save(source_csv_path, target_csv_path)


Progress: 31 summoners processed.
Progress: 61 summoners processed.
Progress: 91 summoners processed.
Progress: 121 summoners processed.
Progress: 151 summoners processed.
Progress: 181 summoners processed.
Progress: 211 summoners processed.
Progress: 241 summoners processed.
Progress: 271 summoners processed.
Progress: 301 summoners processed.
Progress: 331 summoners processed.
Progress: 361 summoners processed.
Progress: 391 summoners processed.
Progress: 421 summoners processed.
Progress: 451 summoners processed.
Progress: 481 summoners processed.
Progress: 511 summoners processed.
Progress: 539 summoners processed.
Progress: 570 summoners processed.
Progress: 601 summoners processed.
Progress: 631 summoners processed.
Progress: 661 summoners processed.
Progress: 691 summoners processed.
Progress: 721 summoners processed.
Progress: 751 summoners processed.
Progress: 781 summoners processed.
Progress: 811 summoners processed.
Final Progress: 814 summoners processed. Task complete.


In [49]:
import asyncio
import aiohttp
import csv
import os
from aiohttp import ClientSession

async def fetch_match_ids(session, url, csv_writer, sem):
    async with sem, session.get(url) as response:
        if response.status == 200:
            match_ids = await response.json()
            for match_id in match_ids:
                csv_writer.writerow([match_id])
        else:
            print(f"Error: {response.status}")

async def run(source_csv_path, target_csv_path, api_key):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }
    
    # Semaphore to control concurrency, aiming for 30 requests per second
    sem = asyncio.Semaphore(30)

    # Ensure the target file directory exists
    os.makedirs(os.path.dirname(target_csv_path), exist_ok=True)

    async with ClientSession(headers=headers) as session, open(target_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
        csv_writer = csv.writer(outfile)
        csv_writer.writerow(['MatchID'])  # Header for target CSV

        tasks = []
        with open(source_csv_path, mode='r', encoding='utf-8') as infile:
            csv_reader = csv.DictReader(infile)
            for row in csv_reader:
                puuid = row['summonerName']  # Assuming puuid is provided in this column
                url = f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count=20"
                tasks.append(fetch_match_ids(session, url, csv_writer, sem))
                
                # Rate limit control: Wait to keep under 30 requests per second
                if len(tasks) >= 30:
                    await asyncio.gather(*tasks)
                    tasks = []
                    await asyncio.sleep(1)  # Pause to respect the rate limit

            # Ensure all remaining tasks are completed
            if tasks:
                await asyncio.gather(*tasks)

# Example usage
#source_csv_path = '../DataProcessing/TestData/BRONZE_players.csv'
#target_csv_path = '../DataProcessing/TestData/BRONZE_GAMES.csv'
#api_key = "RGAPI-e75b46d3-5e51-4829-b978-68790c3ebf56"

# Start the asynchronous event loop
#asyncio.run(run(source_csv_path, target_csv_path, api_key))


RuntimeError: asyncio.run() cannot be called from a running event loop